In [5]:
from requests import exceptions
import argparse
import requests
import cv2
import os

API_KEY = "API_KEY"
MAX_RESULTS = 100
GROUP_SIZE = 50

# 取得したエンドポイントURL
URL = "エンドポイント"

OUTPUT = "test"
if not os.path.isdir(OUTPUT):
  os.mkdir(OUTPUT)

names = ['ペンギン']

for name in names:

  EXCEPTIONS = set([IOError, FileNotFoundError,
      exceptions.RequestException, exceptions.HTTPError,
      exceptions.ConnectionError, exceptions.Timeout])

  term = name
  headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
  params = {"q": term, "offset": 0, "count": GROUP_SIZE, "imageType":"Photo", "color":"ColorOnly"}

  # make the search
  print("[INFO] searching Bing API for '{}'".format(term))
  search = requests.get(URL, headers=headers, params=params)
  search.raise_for_status()

  # grab the results from the search, including the total number of
  # estimated results returned by the Bing API
  results = search.json()
  estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
  print("[INFO] {} total results for '{}'".format(estNumResults,
      term))

  # initialize the total number of images downloaded thus far
  total = 0

  # loop over the estimated number of results in `GROUP_SIZE` groups
  for offset in range(0, estNumResults, GROUP_SIZE):
      # update the search parameters using the current offset, then
      # make the request to fetch the results
      print("[INFO] making request for group {}-{} of {}...".format(
          offset, offset + GROUP_SIZE, estNumResults))
      params["offset"] = offset
      search = requests.get(URL, headers=headers, params=params)
      search.raise_for_status()
      results = search.json()
      print("[INFO] saving images for group {}-{} of {}...".format(
          offset, offset + GROUP_SIZE, estNumResults))
      # loop over the results
      for v in results["value"]:
          # try to download the image
          try:
              # make a request to download the image
              print("[INFO] fetching: {}".format(v["contentUrl"]))
              r = requests.get(v["contentUrl"], timeout=30)

              # build the path to the output image
              print
              ext = v["contentUrl"][v["contentUrl"].rfind("."):v["contentUrl"].rfind("?") if v["contentUrl"].rfind("?") > 0 else None]
              p = os.path.sep.join([OUTPUT, "{}{}".format(
                  str(total).zfill(8), ext)])

              # write the image to disk
              f = open(p, "wb")
              f.write(r.content)
              f.close()

          # catch any errors that would not unable us to download the
          # image
          except Exception as e:
              # check to see if our exception is in our list of
              # exceptions to check for
              if type(e) in EXCEPTIONS:
                  print("[INFO] skipping: {}".format(v["contentUrl"]))
                  continue
          # try to load the image from disk
          image = cv2.imread(p)

          # if the image is `None` then we could not properly load the
          # image from disk (so it should be ignored)
          if image is None:
              print("[INFO] deleting: {}".format(p))
              os.remove(p)
              continue

          # update the counter
          total += 1

[INFO] searching Bing API for 'EmperorPenguin'
[INFO] 100 total results for 'EmperorPenguin'
[INFO] making request for group 0-50 of 100...
[INFO] saving images for group 0-50 of 100...
[INFO] fetching: https://static.independent.co.uk/s3fs-public/thumbnails/image/2018/01/16/10/emperor-penguin.jpg
[INFO] fetching: http://3.bp.blogspot.com/-l5WiG6Nci34/UVKfPBediVI/AAAAAAAAABk/48mdp61vr5k/s1600/Emperor-Penguins-Couple.jpg
[INFO] fetching: https://upload.wikimedia.org/wikipedia/commons/a/a3/Aptenodytes_forsteri_-Snow_Hill_Island%2C_Antarctica_-adults_and_juvenile-8.jpg
[INFO] fetching: http://www.antarctica.gov.au/__data/assets/image/0011/147737/varieties/antarctic.jpg
[INFO] fetching: https://fsmedia.imgix.net/5a/6a/7f/21/f7ab/4c18/b150/49f10bce6a2b/emperor-penguins-famously-raise-one-offspring-at-a-time-in-pairs.jpeg?dpr=2&auto=format%2Ccompress&w=650
[INFO] fetching: http://www.antarctica.gov.au/__data/assets/image/0018/191250/varieties/antarctic.jpg
[INFO] fetching: http://www.antarct

NotADirectoryError: ignored